### LCEL Deepdive

Install required dependencies for LangChain, Groq API integration, and vector storage.

In [ ]:
!pip install -qU langchain-groq langchain_community langchain_huggingface faiss-cpu

Set up environment variables and import necessary LangChain components for building chains with the Groq LLM.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
#from dotenv import load_dotenv
#load_dotenv()
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

Create a simple LCEL chain: prompt → model → output parser. This demonstrates the pipe operator (|) chaining components together.

In [ ]:
prompt = ChatPromptTemplate.from_template("Explain about {topic} in detail")
#model = ChatOpenAI()
# Initialize the Groq model
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

Inspect the prompt template output to see how variables are formatted before being passed to the model.

In [ ]:
print(prompt.invoke({"topic": "ice cream"}))

Invoke the model directly with chat messages to generate a response.

In [ ]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

### What is this "|" in Python?

Implement a custom Runnable class to understand how the pipe operator (|) works. This creates a chain pattern where each component processes data sequentially.

In [ ]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



Define concrete Runnable implementations (AddTen, MultiplyByTwo, ConvertToString) to demonstrate data transformation through a chain.

In [ ]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

Create a chain by composing three custom Runnables using the pipe operator.

In [ ]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

Execute the chain and observe how data flows through each component sequentially.

In [ ]:
result = chain.invoke(10)
print(result)

### Runnables from LangChain

Import LangChain's built-in Runnable types: RunnablePassthrough (passes data through unchanged), RunnableLambda (wraps functions), and RunnableParallel (processes multiple branches).

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

Chain multiple RunnablePassthrough instances together, demonstrating that passthrough components return data unchanged.

In [ ]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

Define a lambda function that converts input strings to uppercase.

In [ ]:
def input_to_upper(input: str):
    output = input.upper()
    return output

Chain RunnablePassthrough with RunnableLambda to apply the uppercase transformation.

In [ ]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

Create a RunnableParallel that simultaneously processes the same input through multiple branches, each stored as a key in the output dictionary.

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

Execute the parallel chain with a simple string input.

In [ ]:
chain.invoke("hello")

Execute the parallel chain with a dictionary input to see how RunnablePassthrough handles structured data.

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

Create a RunnableParallel with a lambda function that extracts a specific key from the input dictionary.

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

Execute the parallel chain, demonstrating selective extraction of input values.

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

### Nested chains - now it gets more complicated!

Define a lambda function that extracts and transforms a specific key from the input dictionary.

In [ ]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

Create a complex nested chain with RunnableParallel containing a sequential pipe (RunnablePassthrough | RunnableLambda) in one branch and a lambda in another.

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

Execute the nested chain to show how complex transformations combine parallel and sequential processing.

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

Define helper functions for assignment and multiplication, then create a RunnableParallel as a base for further chaining.

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(input):
    return 100

def multiply(input):
    return input * 10

Execute the parallel chain with dictionary input.

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

Use the .assign() method to add new computed fields to the chain output while preserving existing data.

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

Execute the chain with assignment to display the combined result with both original and computed values.

In [ ]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

### Combine multiple chains (incl. coercion)

Create extraction and transformation functions, then compose them into a new chain that extracts data and applies uppercase conversion.

In [ ]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

Execute the extraction and transformation chain on a dictionary input.

In [ ]:
new_chain.invoke({"extra": "test"})

Combine the complex nested chain with the extraction chain to demonstrate chain coercion and composition.

In [ ]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

### Real World Example

Build a Retrieval-Augmented Generation (RAG) chain: create embeddings, initialize a vector store with sample data, set up a retriever, and chain it with a prompt and LLM for context-aware question answering.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=embeddings
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | ChatGroq(model="llama-3.1-8b-instant", temperature=0, max_retries=2)
    | StrOutputParser()
)

Query the RAG chain with a question to generate an answer based on retrieved context.

In [ ]:
rag_chain.invoke("What do cats like to eat?")

Debug the RAG chain by inspecting the output of the RunnableParallel component to see the retrieved context and question.

In [ ]:
RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()}).invoke("What do cats like to eat?")

Inspect the formatted prompt with injected context and question values.

In [ ]:
prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"})

Execute the model on the formatted prompt to see the final LLM response before parsing.

In [ ]:
model.invoke(prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"}))